# COMS W1002 Computing in Context: Computational Finance  
## Professor Karl Sigman
## Option Pricing Using Monte Carlo Simulation: Solutions

CF Assignment 2: Solutions
by John Mathews (jjm2212)

__Problem 1:__ For the values of $p=0.5$ and $p=0.75$ obtain an estimate of $p$ by using Monte Carlo Simulation with $n=100, n=1000, n=10,000$ You will generate $n$ _iid_ Bernoulli $(p)$ random variables $B_1, ..., B_n$ and use as the estimate $$ \hat{p}= \frac{1}{n}\sum_{i=1}^{n}B_i $$
This is to show you (convince you) how accurate the Monte Carlo method can be, and how the Strong Law of Large Numbers (SLLN) works in practice.

In [6]:
import random

# Note, we will use this bernoulli function in the future problems as well
def bernoulli(p):
    if random.random() < p:
        return 1
    else:
        return 0

# Basic monte carlo setup
def monte_carlo(p, n):
    count=0
    for i in range(n):
        if bernoulli(p) == 1:
            count += 1
    return count/n

probs = [.5, .75]
n = [100, 1000, 10000]

for prob in probs:
    print("Probability: %s" % prob)
    for num_iter in n:
        print("\t%s trials: %s" %(num_iter, monte_carlo(prob, num_iter)))


Probability: 0.5
	100 trials: 0.49
	1000 trials: 0.497
	10000 trials: 0.5001
Probability: 0.75
	100 trials: 0.72
	1000 trials: 0.765
	10000 trials: 0.7435


__Problem 2:__ Recall the formula for computing the price $C_0$ of an option (derivative of the BLM stock prices) That yields a payoff at time $T$, denote by $C_T$:
$$ C_0 = \frac{1}{(1+r)^T}E^*(C_T), $$  

where $*$ refers to the fact that we must use the value $p^*$ instead of the original (real) $P$ for the up/down probability of the BLM. (The real value of $P$ is not needed for priciing.) Also recall that for $C_T = (S_T - K)^+$, the European call option, the expected value, $E^*(S_T-K)^+$ can be computed explicitly yielding the famous Black-Scholes-Merton option pricing formula:  
$$ C_0 = \frac{1}{(1+r)^T}\sum_{k=0}^{T} \left( \begin{array}{c}
T \\ k  \end{array} \right )(p^*)^k(1-p^*)^{T-k}(u^k d^{T-k}S_0-K)^+ . $$  

You are to use this formula to exactly obtain the price on the one hand, and then use Monte Carlo simulation on the other hand to compare and thus see how accurate the Monte Carlo method can be.

Here are the parameters to use: $T = 10, r = 0.05, u = 1.15, d= 1.01, S_0 = 50, K = 70$. Recall that
$$ p^* = \frac{1+r-d}{u-d} . $$  

For the Monte Carlo, use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging) to see how it gets more accurate as $n$ increases.

In [7]:
import random
from math import factorial

def n_choose_k(n, k):
    return factorial(n)/(factorial(k)*factorial(n-k))

total = 0
T = 10
r = .05
u = 1.15
d = 1.01
S0 = 50
K = 70

def exact_price(T,r,u,d,S0,K):
    total = 0
    p_star= (1+r-d)/(u-d)
    for k in range(T+1):
        S = S0*u**k*d**(T-k)
        total += n_choose_k(T, k)*p_star**k*(1-p_star)**(T-k)*max(S-K, 0)
    total *= (1+r)**-T
    return total

print("Exact Option Value: %s\n" % exact_price(T,r,u,d,S0,K))


def monte_carlo_european_call(n, t, r, u, d, s_o, k):
    discount = 1/((1+r)**t) #discount rate
    trial_sums = 0
    p_star = (1 + r - d) / (u - d)

    for j in range(n): #iterate through the n trials
        temp = 0
        increased = 0 # number of times stock increased

        for i in range(t): #iterate through the i time units
            if bernoulli(p_star) == 1:
                increased += 1

        temp = discount*(((u**increased)*(d**(t-increased))*(s_o))-k)

        trial_sums += max(temp, 0) # We ensure we aren't adding negative numbers to the total

    return trial_sums/n #return the average of the positive payoffs over the n trials.


n = [100, 1000, 100000]
for num_iter in n:
    print("%s european call option monte-carlo trials:" % num_iter)
    print(monte_carlo_european_call(num_iter, T, r, u, d, S0, K), "\n")



Exact Option Value: 7.943399485843426

100 european call option monte-carlo trials:
8.422508727633861 

1000 european call option monte-carlo trials:
7.77356724175302 

100000 european call option monte-carlo trials:
7.921680110469083 



In [8]:
import random

t = 10
r = .05
u = 1.15
d = 1.01
s_o = 50
k = 70

def monte_carlo_asian_call(n, t, r, u, d, s_o, k):
    discount = 1 / ((1 + r)**t) #rate to discount the result by
    p_star = (1 + r - d) / (u - d)
    total = 0

    for i in range(n):
        sum_prices = 0
        price = s_o
        for i in range(t):
            if bernoulli(p_star) == 1:
                price=price*u
            else:
                price=price*d
            sum_prices += price
        val = (sum_prices / t) - k #difference between the average stock price and the strike price.
        total += max(val, 0)
    return (total * discount)/n

n = [100, 1000, 100000]
for num_iter in n:
    print("Asian call option price estimation using monte-carlo simulation for %s trials:" % num_iter)
    print(monte_carlo_asian_call(num_iter, t, r, u, d, s_o, k), "\n")


Asian call option price estimation using monte-carlo simulation for 100 trials:
1.27530670500222 

Asian call option price estimation using monte-carlo simulation for 1000 trials:
0.960366001756742 

Asian call option price estimation using monte-carlo simulation for 100000 trials:
1.1275858716478169 

